In [2]:
import torch
torch.cuda.empty_cache()


In [42]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [43]:
import guidance
lm = model + "Who won the last Kentucky derby and by how much?"
from guidance import gen

lm + gen(max_tokens=10)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [45]:
lm + '''\
Q: Who won the last Kentucky derby and by how much?
A:''' + gen(stop="Q:", max_tokens=50)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [47]:
query = "Who won the last Kentucky derby and by how much?"
lm = model + f'''\
Q: {query}
A: {gen(name="answer", stop="Q:", max_tokens=50)}'''

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [48]:
lm["answer"]


'The 149th running of the Kentucky Derby was held on May 4, 2024, at Churchill Downs in Louisville, Kentucky. The winner was **Mage**, ridden by jockey **Flavien Prat** and'

In [50]:
import guidance

@guidance
def qa_bot(lm, query):
    lm += f'''\
    Q: {query}
    A: {gen(name="answer", stop="Q:")}'''
    return lm

query = "Who won the last Kentucky derby and by how much?"
model + qa_bot(query)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [51]:
lm["answer"]


'The 149th running of the Kentucky Derby was held on May 4, 2024, at Churchill Downs in Louisville, Kentucky. The winner was **Mage**, ridden by jockey **Flavien Prat** and'

In [58]:
from guidance import select

lm = model + f'''\
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.
Choice: {select(["SEARCH", "RESPOND"], name="choice")}'''

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [59]:
lm['choice']

'SEARCH'

In [81]:
@guidance
def qa_bot(lm, query):
    lm += f'''\
    Q: {query}
    Now I will choose to either SEARCH the web or RESPOND.
    Choice: {select(["SEARCH", "RESPOND"], name="choice")}
    '''
    if lm["choice"] == "SEARCH":
        lm += "A: I don't know, Google it!"
    else:
        lm += f'A: {gen(stop="Q:", name="answer")}'
    return lm

lm = model + qa_bot(query)

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [85]:
hasattr(lm, "choice")

False

In [86]:

lm['choice']

'SEARCH'

In [75]:
vars(lm)

{'echo': True,
 '_interpreter': <guidance.models._engine._interpreter.EngineInterpreter at 0x14db320a7cd0>,
 '_active_blocks': {},
 'sampling_params': {},
 '_parent': <guidance.models._transformers.Transformers at 0x14dafa34a3d0>,
 '_parent_id': 1360,
 '_id': 1361,
 '_trace_nodes': {1352:[LiteralInput:"A: I don't know, Google it!"]:[TokenOutput:'A':True:75.94385147094727:Token(token='A', bytes=b'A', prob=nan, masked=False)],
  1353:[]:[TokenOutput:':':True:75.94385147094727:Token(token=':', bytes=b':', prob=0.27966225147247314, masked=False)],
  1354:[]:[TokenOutput:' I':True:75.94385147094727:Token(token=' I', bytes=b' I', prob=0.016418831422924995, masked=False)],
  1355:[]:[TokenOutput:' don':True:75.94385147094727:Token(token=' don', bytes=b' don', prob=0.00017329071124549955, masked=False)],
  1356:[]:[TokenOutput:"'t":True:75.94385147094727:Token(token="'t", bytes=b"'t", prob=0.9999969005584717, masked=False)],
  1357:[]:[TokenOutput:' know':True:75.94385147094727:Token(token=' k

In [76]:
dir(lm)

['__and__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iand__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rsub__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 'add',
 'clear',
 'copy',
 'difference',
 'difference_update',
 'discard',
 'intersection',
 'intersection_update',
 'isdisjoint',
 'issubset',
 'issuperset',
 'pop',
 'remove',
 'symmetric_difference',
 'symmetric_difference_update',
 'union',
 'update']

In [88]:
lm = model + f'''\
Q: {query}
Now I will choose to either SEARCH the web or RESPOND.
Choice: {select(["SEARCH", "RESPOND"], name="choice")}
'''
if lm["choice"] == "SEARCH":
    lm += "Here are 3 search queries:\n"
    for i in range(3):
        lm += f'''{i+1}. "{gen(stop='"', name="queries", temperature=1.0, list_append=True)}"\n'''

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [89]:
lm["queries"]

['last kentucky derby winner and margin of victory',
 'who won the kentucky derby 2023',
 'kentucky derby 2023 final results']

In [90]:
# you can create and guide multi-turn conversations by using a series of role tags
@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        Who are 3 world-class experts (past or present) who would be great at answering this?
        Please don't answer the question or comment on it yet."""

    with assistant():
        lm += gen(name='experts', max_tokens=300)
    
    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
        In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
        If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
        Please start your answer with ANSWER:"""
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)

    return lm
                   
lm = model + experts(query='What is the meaning of life?')

StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [ ]:
# for part in model.stream() + experts(query='What is the meaning of life?'):
# #     print(part)

In [2]:

import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field
from typing import Optional
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json


In [30]:
from pydantic import BaseModel


transfer_passenger_count = 30

condition = [
    # {"charge_name": "transfer passenger charge", "transfer_passenger_count": transfer_passenger_count, "period": "summer airline scheduling season", "rate": "?"},
    # # {"charge_name": "transfer passenger charge", "transfer_passenger_count": transfer_passenger_count, "period": "winter airline scheduling season", "rate": "?"},
    {"charge_name": "runway landing and takeoff charge", "period": "summer airline scheduling season", "atm": "landing", "per tonne MTOW": 1, "rate": "?"},
    # # {"charge_name": "runway landing and takeoff charge", "period": "summer airline scheduling season", "atm": "takeoff", "per tonne MTOW": 1, "rate": "?"},
    # # {"charge_name": "runway landing and takeoff charge", "period": "winter airline scheduling season", "atm": "landing", "per tonne MTOW": 1, "rate": "?"},
    # {"charge_name": "runway landing and takeoff charge", "period": "winter airline scheduling season", "atm": "takeoff", "per tonne MTOW": 1, "rate": "?"}
]


In [4]:

MODEL_ID = "Qwen/Qwen3-30B-A3B"
import torch, outlines
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1️⃣  HF loads & shards the model – ONE LINE does the heavy lifting.
hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",                # shards across all visible GPUs
    low_cpu_mem_usage=True,           # avoids a large RAM peak
)

# 2️⃣  Wrap in Outlines.
tok        = AutoTokenizer.from_pretrained(MODEL_ID)
# model      = outlines.from_transformers(hf_model, tok)          # note new API


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
import guidance
model = guidance.models.Transformers(hf_model, tok)

gpustat is not installed, run `pip install gpustat` to collect GPU stats.


In [6]:
model

In [7]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json

# {'type': 'VALUE',
#  'value': 3.9,
#  'description': 'Transfer passenger charge rate in pounds'}

# 3. Define the expression tree structure
class Op(str, Enum):
    ADD = "ADD"
    MULTIPLY = "MULTIPLY"

class ValueNode(BaseModel):
    """A leaf node with a numeric value"""
    type: str = "VALUE"
    value: float
    description: str = Field(description="Explanation of what this value represents")

class OpNode(BaseModel):
    """An operation node with two children"""
    type: str = "OPERATION"
    operator: Op
    left: "Node"  # Must have both children
    right: "Node"

# Use discriminated union to ensure the model follows one of the valid patterns
class Node(RootModel):
    """Either a value node or an operation node"""
    root: Union[ValueNode, OpNode]
    
    # model_config = dict(extra="forbid")

# # Examples of valid expression trees
# EXAMPLE_VALUE_NODE = ValueNode(
#     type="VALUE", 
#     value=3.9,
#     description="Transfer passenger charge rate in pounds"
# )

# EXAMPLE_OP_NODE = OpNode(
#     type="OPERATION",
#     operator=Op.MULTIPLY,
#     left=Node(root=ValueNode(
#         type="VALUE", 
#         value=3.9, 
#         description="Transfer passenger charge per passenger"
#     )),
#     right=Node(root=ValueNode(
#         type="VALUE", 
#         value=30.0, 
#         description="Number of transfer passengers"
#     ))
# )

# Rebuild the model to handle the forward reference in the Node class
# Node.model_rebuild()


In [8]:

Node.model_rebuild()


In [9]:

with open("output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md", "r") as f:
    markdown_content = f.read()


In [21]:

# # 6. Define the guidance program using the new chat API
# @guidance
# def create_expression_tree(llm, markdown_content, cond, pydantic_class):
    
#     with system():
#         llm = llm + "You are a world-class algorithm for building expression trees from text. Your goal is to construct a JSON object that represents the calculation logic for a 'rate' based on a document and a set of conditions."

#     with user():
#         llm += f"""Here is the document:
# ---
# {markdown_content}
# ---

# Given the following condition:
# {cond}

# Construct the expression tree for the rate based on the document and condition."""

#     with assistant():
#         llm += gen_json(name="expression_tree", schema=pydantic_class)
    
#     return llm

# 6. Define the guidance program using the new chat API

@guidance
def create_expression_tree(llm, markdown_content, cond, pydantic_class):
    
    with system():
        llm = llm + """You are a world-class algorithm for building expression trees from text. Your goal is to construct a JSON object that represents the calculation logic for a 'rate' based on a document and a set of conditions.
        
You MUST follow the Node schema exactly. It requires either:
1. A ValueNode with 'type': 'VALUE' and a 'value' field containing a number, OR
2. An OpNode with 'type': 'OPERATION', an 'operator' which must be 'ADD' or 'MULTIPLY', and 'left'/'right' fields containing other nodes.

Here are examples of valid expression trees:
1. Simple value: {"type": "VALUE", "value": 3.9, "description": "Transfer passenger charge rate in pounds"}
2. Simple multiplication: {"type": "OPERATION", "operator": "MULTIPLY", "left": {"type": "VALUE", "value": 3.9, "description": "Transfer passenger charge per passenger"}, "right": {"type": "VALUE", "value": 30.0, "description": "Number of transfer passengers"}}"""

    with user():
        llm += f"""Here is the document:
---
{markdown_content}
---

Given the following condition:
{cond}

Construct the expression tree for the rate based on the document and condition."""

    with assistant():
        # Use stricter constraints for generation
        llm += gen_json(
            name="expression_tree", 
            schema=pydantic_class,
            max_tokens=200,
            # validation_fn=lambda x: pydantic_class.model_validate_json(x) is not None
        )

    return llm


In [11]:
Node

__main__.Node

In [32]:

# 7. Loop over conditions and generate the expression tree
for cond in condition:
    print(f"--- Condition: {cond} ---")
    try:
        # Execute the guidance program
        result_lm = model + create_expression_tree(markdown_content=markdown_content, cond=cond, pydantic_class = Node)
        print(result_lm)
        # Extract and validate the generated JSON
        # The output from the new API is already a Pydantic object, so no need to parse
        expression_tree = result_lm["expression_tree"]
        # print(expression_tree.model_dump_json(indent=2))
        # cond["rate"] = expression_tree
        print(expression_tree)

    except Exception as e:
        print(f"An error occurred: {e}")
        cond["rate"] = None

# 8. Print the final conditions with expression trees
# print("\n--- Final Result ---")
# print(condition)


--- Condition: {'charge_name': 'runway landing and takeoff charge', 'period': 'summer airline scheduling season', 'atm': 'landing', 'per tonne MTOW': 1, 'rate': '?'} ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [33]:
expression_tree

'{"type": "VALUE", "value": 5.5, "description": "Band 1 rate for summer season"}'

In [34]:
loaded_json = json.loads(expression_tree)

In [35]:
loaded_json

{'type': 'VALUE', 'value': 5.5, 'description': 'Band 1 rate for summer season'}

In [36]:
print(json.dumps(loaded_json, indent=4))

{
    "type": "VALUE",
    "value": 5.5,
    "description": "Band 1 rate for summer season"
}


In [37]:
result = Node.model_validate_json(expression_tree)

In [38]:
print(result.model_dump_json(indent=8))

{
        "type": "VALUE",
        "value": 5.5,
        "description": "Band 1 rate for summer season"
}


In [29]:
expression_tree

'{"type": "VALUE", "value": 3.9, "description": "Transfer passenger charge rate in pounds"}'

In [17]:


generator = outlines.generate.json(model, Node)


/tmp/ipykernel_825144/440850722.py:1: DeprecationWarning: 
        The `json` function is deprecated starting from v1.0.0.
        Do not use it. Support for it will be removed in v1.1.0.
        Use the `Generator` object instead:
        ```python
        from outlines import Generator
        from outlines.types import JsonSchema
        schema_str = '...'  # JSON Schema as a string
        generator = Generator(model, JsonSchema(schema_str))
        ```
        You can then call the generator created with a prompt to generate
        JSON data that matches the schema.
        
  generator = outlines.generate.json(model, Node)


KeyboardInterrupt: 

In [ ]:

# 6️⃣ Evaluation function for the expression tree
def evaluate(node: Node, context: dict) -> float:
    if node.value is not None:
        if node.value.literal is not None:
            return node.value.literal
        if node.value.variable is not None:
            return context.get(node.value.variable, 0)
        return 0

    if node.operator and node.childA and node.childB:
        val_a = evaluate(node.childA, context)
        val_b = evaluate(node.childB, context)

        if node.operator == Op.ADD:
            return val_a + val_b
        if node.operator == Op.MULTIPLY:
            return val_a * val_b
        if node.operator == Op.GREATER_THAN:
            return val_a > val_b
        if node.operator == Op.IF:
            # childA is condition, childB is true-branch, childC is false-branch
            if val_a:
                return val_b
            elif node.childC:
                return evaluate(node.childC, context)
    return 0



In [ ]:
# 7️⃣ Loop over conditions, generate and evaluate the tree
for cond in condition:
    prompt = f"""You are a world-class algorithm for parsing rules from text into a calculation tree.

    Here is a document:
    ---
    {markdown_content}
    ---

    Given the following condition:
    {cond}

    Create a JSON object that represents the calculation logic for the charge as an expression tree. 
    - Use 'IF' for conditional logic.
    - Use 'GREATER_THAN' for comparisons.
    - Use 'MULTIPLY' and 'ADD' for arithmetic.
    - Use 'value.literal' for numbers found in the text.
    - Use 'value.variable' to reference keys from the condition, like 'per tonne MTOW'.
    """
    tree = generator(prompt)
    cond["rate"] = evaluate(tree, cond)


In [ ]:

# 8️⃣ Print the updated conditions
import json
print(json.dumps(condition, indent=2))
